In [ ]:
import pandas as pd

import duckdb

con = duckdb.connect(database=":memory:")


In [ ]:
model_name = "ja_ginza_electra"
db = pd.read_csv(f"../data/ted_npvs_{model_name}.csv")
db = pd.concat([db, pd.read_csv(f"../data/jnlp_npvs_{model_name}.csv")])
# NPV＋コーパスごとに集計したいので，Pandasのvalue_counts()を利用し，その結果 (Seriesオブジェクト) をDataFrameに戻す
db = db.value_counts().to_frame(name="frequency").reset_index()
db


In [ ]:
# コーパス間の頻度が比べるために一番小さいコーパスのNPVの数で正規化する
corpus_freqs: dict[str, int] = {
    corpus: db[db.corpus == corpus]["frequency"].sum() for corpus in db.corpus.unique()
}
min_count = min(corpus_freqs.values())
min_count

In [ ]:
corpus_norm = {
    corpus: min_count / frequency for corpus, frequency in corpus_freqs.items()
}

print(corpus_norm)

In [ ]:
db["norm_frequency"] = db.apply(
    lambda r: r["frequency"] * corpus_norm[r["corpus"]], axis=1
)
db


In [ ]:
db[db.n == "変化"].drop(columns=["n"]).to_dict("records")

In [ ]:
con.execute(
    """
SELECT * FROM db
WHERE n = ?
GROUP BY p, n, v, corpus, frequency, norm_frequency
ORDER BY p, norm_frequency DESC, n
""",
    ["変化"],
).df()

各種の共起尺度の計算：
-   [Relational cooccurrences and contingency tables](http://collocations.de/AM/index.html)